In [1]:
import pandas as pd
import numpy as np

In [7]:
data2014 = pd.read_csv("Output/2014dataset.csv",low_memory=False)
data2015 = pd.read_csv("Output/2015dataset.csv",low_memory=False)
# data2016 = pd.read_csv("Output/2016dataset.csv",low_memory=False) doesn't exist yet

In [9]:
states = data2014["STATE"].drop_duplicates().as_matrix()
print(states)
    
#print("state,individualratio,smallgroupratio,individualtransfers,smallgrouptransfers,individualmm,smallgroupmm")
#for state in states:
#    temp = merged2015.where(merged2015["STATE"] == state).dropna(how='all')
#    a = sum(abs(temp["HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)"]).convert_objects(convert_numeric=True).dropna())
#    b = sum(abs(temp["HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP MARKET)"]).convert_objects(convert_numeric=True).dropna())
#    c = sum(temp["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"].convert_objects(convert_numeric=True).dropna())
#    d = sum(temp["7.4Membermonths7HealthInsuranceSMALLGROUPTotalasof3/31/15"].convert_objects(convert_numeric=True).dropna())
#    individual_ratio = a/c
#    smallgroup_ratio = b/d
#    print(state + "," + str(individual_ratio) + "," + str(smallgroup_ratio) + "," +
#          str(a) + "," + str(b) + "," + str(c) + "," + str(d))

['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID' 'IL'
 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC' 'ND' 'NE'
 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD' 'TN' 'TX' 'UT'
 'VA' 'WA' 'WI' 'WV' 'WY']


In [116]:
# weighted rigorous statistics
# group can be i for individual or sg for small group
# absval can be 1 for yes and 0 for no

# functions:
# def getmm(group) - returns member months column
# xbarybar(dataset, column_name, state, group, absval) - returns weighted avg of a column
# sigmaxsigmay(dataset, column_name, state, group, absval) - returns weighted std of a column
# covxy(dataset, column_name1, column_name2, state, group, absval1, absval2) - returns weighted cov of two columns
# covspecial(dataset, column_name1, yi, ybar, state, group, absval1) - returns weighted cov of one column and one state measure
# r(dataset, column_name1, column_name2, state, group, absval1, absval2) - returns weighted r of two columns
# rspecial(dataset, column_name1, yi, ybar, sigmay, state, group, absval1) - returns weighted r of one column and one state measure
# rsquared(dataset, column_name1, column_name2, state, group, absval1, absval2) - returns weighted r^2 of two columns

def getmm(group):
    if group == "i":
        return "7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"
    elif group == "sg":
        return "7.4Membermonths7HealthInsuranceSMALLGROUPTotalasof3/31/15"
    else:
        return ""

def xbarybar(dataset, column_name, state, group, absval):
    temp = dataset #.where(dataset["STATE"] in states).dropna(how='all')
    mm = getmm(group)
    #print(temp["1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15"])
    sum_mm = sum(temp[mm].convert_objects(convert_numeric=True).dropna())
    #print(sum(temp[column_name].dropna()) / 19)
    #print(temp[[mm, column_name]])
    if sum_mm == 0:
        return
    numerator = 0
    denominator = 0
    for index, row in temp.iterrows():
        if (pd.notnull(row[column_name])) and (pd.notnull(row[mm])):
            ratio = row[mm] / sum_mm
            denominator = denominator + ratio
            if absval == 1:
                numerator = numerator + (abs(row[column_name]) * ratio)
            else:
                numerator = numerator + (row[column_name]  * ratio)
    return numerator / denominator

def sigmaxsigmay(dataset, column_name, state, group, absval):
    temp = dataset #.where(dataset["STATE"] in states).dropna(how='all')
    mm = getmm(group)
    sum_mm = sum(temp[mm].convert_objects(convert_numeric=True).dropna())
    if sum_mm == 0:
        return "-1"
    numerator = 0
    denominator = 0
    xbar = xbarybar(dataset, column_name, state, group, absval)
    for index, row in temp.iterrows():
        if (pd.notnull(row[column_name])) and (pd.notnull(row[mm])):
            ratio = row[mm] / sum_mm
            denominator = denominator + ratio
            if absval == 1:
                numerator = numerator + ((abs(row[column_name]) - abs(xbar))**2 * ratio)
            else:
                numerator = numerator + ((row[column_name] - xbar)**2 * ratio)
    return np.sqrt(numerator / denominator)

def covxy(dataset, column_name1, column_name2, state, group, absval1, absval2):
    temp = dataset #.where(dataset["STATE"] in states).dropna(how='all')
    mm = getmm(group)
    sum_mm = sum(temp[mm].convert_objects(convert_numeric=True).dropna())
    if sum_mm == 0:
        return -1
    numerator = 0
    denominator = 0
    xbar = xbarybar(dataset, column_name1, state, group, absval1)
    ybar = xbarybar(dataset, column_name2, state, group, absval2)
    for index, row in temp.iterrows():
        if (pd.notnull(row[column_name2])) and (pd.notnull(row[column_name1])) and (pd.notnull(row[mm])):
            xi = row[column_name1]
            yi = row[column_name2]
            ratio = row[mm] / sum_mm
            denominator = denominator + ratio
            if absval1 == 1 and absval2 == 1:
                numerator = numerator + ((abs(xi) - xbar) * (abs(yi) - ybar) * ratio)
            elif absval1 == 1:
                numerator = numerator + ((abs(xi) - xbar) * (yi - ybar) * ratio)
            elif absval2 == 1:
                numerator = numerator + ((xi - xbar) * (abs(yi) - ybar) * ratio)
            else:
                numerator = numerator + ((xi - xbar) * (yi - ybar) * ratio)
    return numerator / denominator

def covspecial(dataset, column_name1, yi, ybar, state, group, absval1):
    temp = dataset #.where(dataset["STATE"] == state).dropna(how='all')
    mm = getmm(group)
    sum_mm = sum(temp[mm].convert_objects(convert_numeric=True).dropna())
    if sum_mm == 0:
        return "-1"
    numerator = 0
    xbar = xbarybar(dataset, column_name1, state, group, absval1)
    for index, row in temp.iterrows():
        if (pd.notnull(row[column_name1])) and (pd.notnull(row[mm])):
            xi = row[column_name1]
            if absval1 == 1:
                numerator = numerator + ((abs(xi) - xbar) * (yi - ybar) * row[mm])
            else:
                numerator = numerator + ((xi - xbar) * (yi - ybar) * row[mm])
    return numerator / sum_mm

def r(dataset, column_name1, column_name2, state, group, absval1, absval2):
    cov = covxy(dataset, column_name1, column_name2, state, group, absval1, absval2)
    sigx = sigmaxsigmay(dataset, column_name1, state, group, absval1)
    sigy = sigmaxsigmay(dataset, column_name2, state, group, absval2)
    print(cov)
    print(sigx)
    print(sigy)
    return cov / (sigx * sigy)

def rspecial(dataset, column_name1, yi, ybar, sigmay, state, group, absval1):
    cov = covspecial(dataset, column_name1, yi, ybar, state, group, absval1)
    sigx = sigmaxsigmay(dataset, column_name1, state, group, absval1)
    return cov / (sigx * sigmay)
    

def rsquared(dataset, column_name1, column_name2, state, group, absval1, absval2):
    return r(dataset, column_name1, column_name2, state, group, absval1, absval2)**2

In [117]:
r(data2014, "1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15",
 "1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15",
 "CA", "i", 0, 1)

C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


8.13538075723e+17
901963515.929
901963395.92


0.99999999987667576

In [118]:
r(data2014, "1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15",
 "HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)",
 "CA", "i", 0, 1)

C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


3.08496497894e+16
901963515.929
59900721.3696


0.57099093677738655

In [119]:
r(data2014, "2.16Totalincurredclaims2HealthInsuranceINDIVIDUALTotalasof3/31/15",
 "HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)",
 "CA", "i", 0, 1)

C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


2.53560613158e+16
944917130.121
59900721.3696


0.44797731097056237

In [120]:
r(data2015, "2.16Totalincurredclaims2HealthInsuranceINDIVIDUALTotalasof3/31/15",
 "HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)",
 "CA", "i", 0, 1)

C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


3.18088368603e+16
1289100245.96
73036114.6159


0.33784962093568549

In [121]:
r(data2015, "1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15",
 "HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)",
 "CA", "i", 0, 1)

C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


3.47218822056e+16
1133323927.55
73036114.6159


0.41948027100652396